In [1]:
import re
import numpy as np

targets = ["probability", "test_loss", "avg_train_loss", "avg_abs_test_loss"]
target = targets[3]
n = 50
k = 5

In [2]:
def extract_actual_ranks(file_content):
    general_ranks = re.findall(r'Actual rank: (\d+)', file_content)
    result = {
        "IWLS": int(general_ranks[0]),
        "Adaptive IWLS": int(general_ranks[1]),
        "Margin-based": min(int(general_ranks[2]), int(general_ranks[3])),
        "First-order": int(general_ranks[4])
    }
    return result

In [3]:
seeds = [1, 21, 41, 61, 81, 2, 22, 42, 62, 82, 3, 23, 43, 63, 83, 4, 24, 44, 64, 84]
covs = [0.5, 1.0, 2.0, 5.0]
methods = ["IWLS", "Adaptive IWLS", "Margin-based", "First-order"]

result_array = np.zeros((len(seeds), len(covs), len(methods)), dtype=int)

# Process each file and populate the array
for seed_idx, seed in enumerate(seeds):
    for cov_idx, cov in enumerate(covs):
        file_path = f'results/target={target}/s={seed}_n={n}_k={k}_cov={cov}.txt'

        with open(file_path, 'r') as file:
            file_content = file.read()
            actual_ranks = extract_actual_ranks(file_content)

            # Populate the array
            for method_idx, method_name in enumerate(methods):
                result_array[seed_idx, cov_idx, method_idx] = actual_ranks.get(method_name, 0)  # Default to 0 if method not found

# Save result_array to a binary file
np.save(f'results/target={target}/result.npy', result_array)

# Load result_array back from the file
result_array = np.load(f'results/target={target}/result.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'results/target=avg_abs_test_loss/s=2_n=50_k=5_cov=2.0.txt'

In [ ]:
def Borda_count(result_array, weights=[4, 3, 2, 1]):
    num_seeds, num_covs, num_methods = result_array.shape

    # Initialize weighted Borda count array
    weighted_borda_count = np.zeros((num_seeds, num_covs, num_methods), dtype=int)

    # Calculate weighted Borda count for each seed and covariance
    for seed_idx in range(num_seeds):
        for cov_idx in range(num_covs):
            # Sort indices based on actual ranks for the current seed and covariance
            sorted_indices = np.argsort(result_array[seed_idx, cov_idx])

            # Assign weighted Borda count scores
            for rank, method_idx in enumerate(sorted_indices):
                weighted_borda_count[seed_idx, cov_idx, method_idx] = weights[rank]

    # Sum weighted Borda count scores across seeds and covariances
    total_weighted_borda_count = weighted_borda_count.sum(axis=(0, 1))

    # Rank methods based on total weighted Borda count scores
    ranked_methods_weighted = np.argsort(total_weighted_borda_count)[::-1]

    # Display the ranking based on weighted Borda count scores
    print(f"\nWeighted Borda Count Rankings (target={target}):")
    for rank, method_idx in enumerate(ranked_methods_weighted):
        method_name = ["IWLS", "Adaptive IWLS", "Margin-based", "First-order"][method_idx]
        print(f"{rank + 1}. {method_name}: {total_weighted_borda_count[method_idx]}")

In [ ]:
Borda_count(result_array)


Weighted Borda Count Rankings (target=avg_train_loss):
1. IWLS: 232
2. Adaptive IWLS: 220
3. Margin-based: 192
4. First-order: 156


In [ ]:
def hitting_rate(result_array, rank_range=(1, 1)):
    num_seeds, num_covs, num_methods = result_array.shape
    total_samples = num_seeds * num_covs

    hitting_rates = np.zeros(num_methods, dtype=float)

    for method_idx in range(num_methods):
        total_hits = 0
        for seed_idx in range(num_seeds):
            for cov_idx in range(num_covs):
                rank = result_array[seed_idx, cov_idx, method_idx]

                if rank_range[1] == rank_range[0]:
                    total_hits += 1 if rank == rank_range[0] else 0
                else:
                    total_hits += 1 if rank in range(rank_range[0], rank_range[1] + 1) else 0

        hitting_rates[method_idx] = total_hits / total_samples

    ranked_methods_hitting = np.argsort(hitting_rates)[::-1]

    # Display hitting rates
    print(f"\nHitting Rates (range={rank_range}, target={target}):")
    for rank, method_idx in enumerate(ranked_methods_hitting):
        method_name = ["IWLS", "Adaptive IWLS", "Margin-based", "First-order"][method_idx]
        print(f"{rank + 1}. {method_name}: {hitting_rates[method_idx]}")

In [ ]:
hitting_rate(result_array, rank_range=(1, 10))


Hitting Rates (range=(1, 10), target=avg_train_loss):
1. First-order: 0.0
2. Margin-based: 0.0
3. Adaptive IWLS: 0.0
4. IWLS: 0.0
